## 6 - Transformers for Sentiment Analysis

> Trong notebook này, ta sẽ sử dụng transformer model, được giới thiệu lần đầu tiên trong paper [này](https://arxiv.org/pdf/1706.03762.pdf). Đặc biệt, ta sẽ sử dụng mô hình BERT (Bidirectional Encoder Representations from Transformers) từ paper [này](https://arxiv.org/pdf/1810.04805.pdf).

> Transformer models có kích thước lớn hơn rất nhiều các models ở notebooks trước. Ta sẽ sử dụng [thư viện transformer](https://github.com/huggingface/transformers) để lấy pre-trained transformers models và sử dụng chúng như embedding layers của mình. Ta sẽ freeze (không train) transformer và chỉ train phần còn lại của model, no sẽ học từ các biểu diễn sinh ra bởi transformer. Trong trường hợp này, ta sẽ sử dụng *multi-layer bi-directional GRU*, tuy nhiên bất cứ mô hình nào cũng có thể học được từ biểu diễn của transformers.

### Preparing Data

In [1]:
import torch

import random
import numpy as np

SEED = 1234 

random.seed(SEED)
np.random.seed(SEED)    
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

> Transformer được train với một tập vocab cụ thể, điều đó có nghĩa là chúng ta cần train chính xác với tập vocab và cần tokenize data giống với cách mà transformer được tokenize.

> May mắn thay, thư viện transformer có sẵn **tokenizer** cho mỗi transformer model mà nó cung cấp. Trong trường hợp này, chúng ta sử dụng model BERT lower tất cả các từ. Ta load tokenizer bằng cách sử dụng: `bert-base-uncased`.

In [2]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

> `tokenizer` có attribute `vocab` chứa vobulary thực mà ta sẽ sử dụng. Ta cùng tìm hiểu về vocab này.

In [3]:
tokenizer.vocab_size

30522

> Ta sử dụng `tokenizer` như sau:

In [4]:
test_sentence = "This is a test sentence"

tokenized = tokenizer.tokenize(test_sentence)
tokenized

['this', 'is', 'a', 'test', 'sentence']

> Ta còn có thể số hóa các tokens trong vocab bằng cách sử dụng `tokenizer.convert_tokens_to_ids`.

In [5]:
tokenizer.convert_tokens_to_ids(tokenized)

[2023, 2003, 1037, 3231, 6251]

> Transformer còn được train với các tokens đặc biệt dùng để đánh dấu bắt đầu và kết thúc câu, chi tiết tại [đây](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel). Bên cạnh đó, ta còn có thể lấy được các padding và unknow token.

> **NOTE**: tokenizer có thuộc tính bắt đầu sequence và kết thúc sequence (`bos_token` và `eos_token`) tuy nhiên không nên sử dụng chúng trong transformer.

In [6]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


> Ta có thể lấy chỉ số của các tokens đặc biệt này bằng cách sử dụng vocab.

In [7]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


> chúng ta cần phải xử lý thêm 1 vấn đề nữa là model được train trên các sequences với chiều dài tối đa đã được định sẵn nên model sẽ không biết cách xử lý những sequence có chiều dài lớn hơn maximum length đã được chỉ định ở trên. Để kiểm tra maximum length, ta sử dụng `max_model_input_sizes` cho phiên bản transformer mà ta sử dụng:

In [8]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']
print(max_input_length)

512


> Trước đây, ta sử dụng `SpaCy` tokenizer để tokenize các sentences. Tuy nhiên giờ đây ta cần một hàm được truyền vào trường `TEXT` để thực hiện tokenize. hàm đó cần thực hiện thêm cả việc cắt ngắn câu nếu nó vượt quá maximum length cho trước. Lưu ý là maximum length phải trừ đi 2 vì ta cần thêm 2 tokens vào đầu và cuối mỗi sentences.

In [9]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    return tokens

> Tiếp theo, ta thực hiện xây dựng các trường. Transformer yêu cầu batch dimension trước, nên ta set `batch_first = True`. Do ta đã có sẵn vocabulary cho các text, được cung cấp bởi transformer nên ta set `use_vocab = False` để thông báo với torchtext rằng ta tự xử lý vấn đề vocab. Tiếp theo, ta cần truyền hàm `tokenize_and_cut` để làm tokenizer. Đối số `preprocessing` là một hàm nhận đầu vào là các sentences đã được tokenized, đó sẽ là nơi chúng ta chuyển tokens thành các chỉ số tương ứng của chúng. Cuối cùng là các tokens đặc biệt, lưu ý là ta truyền vào chỉ số của chúng chứ không phải bản thân tokens (dạng string).

In [10]:
from torchtext.legacy import data

TEXT = data.Field(batch_first = True,
                    use_vocab = False,
                    tokenize = tokenize_and_cut,
                    preprocessing = tokenizer.convert_tokens_to_ids,
                    init_token = init_token_idx,
                    eos_token = eos_token_idx,
                    pad_token = pad_token_idx,
                    unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

> Load data và tạo train, test, valid set.

In [12]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [13]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


> Kiểm tra 1 instance để đảm bảo nó đã được số hóa.

In [14]:
vars(train_data.examples[6])

{'text': [1996,
  18458,
  1997,
  6644,
  9016,
  4627,
  2066,
  2009,
  2453,
  2031,
  2242,
  2000,
  3749,
  1012,
  1037,
  2177,
  1997,
  2267,
  13496,
  2044,
  4399,
  1006,
  1999,
  1996,
  2991,
  1029,
  1007,
  3632,
  2000,
  1037,
  7001,
  6644,
  1999,
  1996,
  5249,
  2073,
  2028,
  2011,
  2028,
  2027,
  2024,
  4457,
  2011,
  2019,
  16100,
  5771,
  5983,
  7865,
  1012,
  1026,
  7987,
  1013,
  1028,
  1026,
  7987,
  1013,
  1028,
  6854,
  1010,
  1996,
  2034,
  20423,
  2003,
  2073,
  2151,
  6556,
  3787,
  1997,
  2143,
  3737,
  2644,
  1012,
  6644,
  9016,
  2003,
  2210,
  2062,
  2084,
  2267,
  4268,
  2559,
  2005,
  3348,
  1010,
  22017,
  4371,
  1010,
  3331,
  2512,
  1011,
  2644,
  2055,
  2498,
  1010,
  1998,
  3773,
  2129,
  2116,
  1042,
  1011,
  9767,
  2027,
  2064,
  2131,
  2046,
  1015,
  1024,
  2871,
  2781,
  2030,
  2174,
  2146,
  2023,
  6752,
  2003,
  1012,
  1026,
  7987,
  1013,
  1028,
  1026,
  7987,
  1013,
  1

> Ta còn có thể sử dụng `convert_ids_to_tokens` để chuyển từ chỉ số về dạng string.

In [15]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['text'])
tokens

['the',
 'premise',
 'of',
 'cabin',
 'fever',
 'starts',
 'like',
 'it',
 'might',
 'have',
 'something',
 'to',
 'offer',
 '.',
 'a',
 'group',
 'of',
 'college',
 'teens',
 'after',
 'finals',
 '(',
 'in',
 'the',
 'fall',
 '?',
 ')',
 'goes',
 'to',
 'a',
 'resort',
 'cabin',
 'in',
 'the',
 'woods',
 'where',
 'one',
 'by',
 'one',
 'they',
 'are',
 'attacked',
 'by',
 'an',
 'unseen',
 'flesh',
 'eating',
 'virus',
 '.',
 '<',
 'br',
 '/',
 '>',
 '<',
 'br',
 '/',
 '>',
 'unfortunately',
 ',',
 'the',
 'first',
 'paragraph',
 'is',
 'where',
 'any',
 'remote',
 'elements',
 'of',
 'film',
 'quality',
 'stop',
 '.',
 'cabin',
 'fever',
 'is',
 'little',
 'more',
 'than',
 'college',
 'kids',
 'looking',
 'for',
 'sex',
 ',',
 'boo',
 '##ze',
 ',',
 'talking',
 'non',
 '-',
 'stop',
 'about',
 'nothing',
 ',',
 'and',
 'seeing',
 'how',
 'many',
 'f',
 '-',
 'bombs',
 'they',
 'can',
 'get',
 'into',
 '1',
 ':',
 '40',
 'minutes',
 'or',
 'however',
 'long',
 'this',
 'mess',
 'is'

> Bên trên, ta mới chỉ xử lý vocabulary cho text, còn label ta vẫn phải tự xây dựng vocabulary.

In [16]:
LABEL.build_vocab(train_data)

> Tạo iterator.

In [17]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device)
    

### Build the Model

> hãy chắc chắn rằng ta load model cùng với model ta áp dụng tokenizer.

In [18]:
from transformers import BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

Downloading: 100%|██████████| 570/570 [00:00<00:00, 225kB/s]
Downloading: 100%|██████████| 440M/440M [01:09<00:00, 6.32MB/s] 


> Tiếp theo, ta thực hiện xây dựng model. <br>
> Thay vì sử dụng embedding layer để có được word embeddings, ta sẽ sử dụng pre-trained transformer model. Các embedding vectors này sẽ được truyền vào GRU để thực hiện phân tích cảm xúc của các sentences. Chúng ta sẽ lấy embedding dimension size (gọi là `hidden_size`) từ transformer thông qua các attribute của nó.

> Bên trong hàm foward, ta đặt transformer bên trong `no_grad` để đảm bảo phần transformer không được tính đạo hàm. Transformer trả về embeddings cho toàn bộ sequence và một *pooled output*. Tài liệu [này](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel) chỉ ra rằng *pooled output* sẽ kém hơn trong việc tổng hợp ý nghĩa câu đầu vào, tốt hơn hết ta nên lấy trung bình hoặc pooling hidden-states của toàn bộ input sentence, do đó ta không sử dụng *pooled output*. 

In [19]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)



    def forward(self, text):
            
            #text = [batch size, sent len]
                    
            with torch.no_grad():
                embedded = self.bert(text)[0]
                    
            #embedded = [batch size, sent len, emb dim]
            
            _, hidden = self.rnn(embedded)
            
            #hidden = [n layers * n directions, batch size, emb dim]
            
            if self.rnn.bidirectional:
                hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
            else:
                hidden = self.dropout(hidden[-1,:,:])
                    
            #hidden = [batch size, hid dim]
            
            output = self.out(hidden)
            
            #output = [batch size, out dim]
            
            return output

In [20]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [21]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 112,241,409 trainable parameters


> Tổng có 112M tham số, tuy nhiên 110M tham số thuộc transformer (và ta không cần train chúng, chỉ cần train 2M tham số còn lại).

> Để đóng băng các weights (không train chúng), ta cần set `requires_grad = False`. Để thực hiện việc này, ta chỉ cần lặp xuyên suốt `named_parameters`, nếu nó thuộc `bert`, ta đóng băng.

In [22]:
for name, param in model.named_parameters():
    if name.startswith('bert'):
        param.requires_grad = False

In [23]:
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,759,169 trainable parameters


> Oh, bây giờ model chỉ còn 2M7 tham số cần train.

> Chúng ta sẽ kiểm tra tên của các parameters có thể train để chắc chắn.

In [24]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

rnn.weight_ih_l0
rnn.weight_hh_l0
rnn.bias_ih_l0
rnn.bias_hh_l0
rnn.weight_ih_l0_reverse
rnn.weight_hh_l0_reverse
rnn.bias_ih_l0_reverse
rnn.bias_hh_l0_reverse
rnn.weight_ih_l1
rnn.weight_hh_l1
rnn.bias_ih_l1
rnn.bias_hh_l1
rnn.weight_ih_l1_reverse
rnn.weight_hh_l1_reverse
rnn.bias_ih_l1_reverse
rnn.bias_hh_l1_reverse
out.weight
out.bias


### Train the Model

In [25]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [26]:
criterion = nn.BCEWithLogitsLoss()


In [27]:
model = model.to(device)
criterion = criterion.to(device)

In [28]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [29]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [30]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [31]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

> Do model cần truyền data vào transformer và xử lý nên mô hình train khá lâu.

In [32]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), './models/tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')


KeyboardInterrupt: 

In [ ]:
model.load_state_dict(torch.load('./models/tut6-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

### Inference

> Ta thực hiện các bước sau: <br>
>> * Tokenize input sequence
>> * Cắt chúng sau cho có chiều dài nhỏ hơn hoặc bằng maximum length.
>> * Thêm ký tự đặc biệt vào sequence, chuyển sang dạng tensor, tạo batch dimension giả và truyền vào model.

In [ ]:
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [ ]:
predict_sentiment(model, tokenizer, "This film is terrible")


In [ ]:
predict_sentiment(model, tokenizer, "This film is great")


## END